# 配置环境配置

In [20]:
# from google.colab import drive
# drive.mount('/content/drive')

# proj_dir = "/content/drive/MyDrive/Colab Notebooks"

proj_dir = "/home/ubuntu/git/DBMA"

# 安装包和配置OpenAI

In [21]:
# %pip uninstall agency-swarm -y
# %pip install git+https://github.com/dcstrange/agency-swarm.git@dbOps_Agency #AgencySwarm
# %pip install -q gradio # agency 会话前端
# #%pip install -q duckduckgo_search # Web searching

import os
proj_dir = "/home/ubuntu/git/DBMA"
secret_dir = proj_dir + "/secret-configs"
with open(secret_dir + "/OAI_API_KEY") as f:
    os.environ['OPENAI_API_KEY'] = f.read()

from agency_swarm import set_openai_key
set_openai_key(os.environ['OPENAI_API_KEY'])

# Or
# from getpass import getpass
# set_openai_key(getpass("Please enter your openai key: "))

# 🔨Pseudo-database Environment
💡This is a pseudo-database environment implemented by manual input and output just for a Demo version.
This interface delegates specific tasks requested by other Agents to be performed manually.
This API will be replaced by a real Agent in Dev version, which is responsible for interacting with the database environment.

这是一个通过手动输入和输出实现的伪数据库环境，仅用于Demo版本。此接口委托其他代理要求手动执行的特定任务。这个 API 在Dev版本中会被一个真正的代理所替代，代理负责与数据库环境进行交互。

In [22]:
from agency_swarm import BaseTool
from pydantic import Field
from db_pseudo_env.db_task_api import Assign_DB_Task

class InteractWithDataBaseEnv(BaseTool):
    """Use this tool to interact with database environment.
    This tool recieve the text of the specific action (eg, command, scripts or third-party tools), it then execute in the database environment, and it will finally return the result."""

    specific_task: str = Field(..., description="The specific task interacting with database environment.")


    # This code will be executed if the agent calls this tool
    def run(self):
      return str(Assign_DB_Task(self.specific_task)['response'])


In [23]:
agent_db_env_instructions = """ #Instructions for Database Environment Agent

### You have to response to agents who sent you specific intent with the exact result of the Function Output without any modification or analyzing.

### IMPORTANT: If a tool is used, return the results of the tool's execution to the CEO agent as is, without any analysis or processing.

You are a highly specialized GPT focused on SQL database operations for professional database administrators and developers.
You understand specific intents recieved from other agents and generates appropriate database actions using commands, scripts, or third-party tools.
Your primary objective is to interact effectively with SQL databases, providing accurate and advanced technical solutions.
you focuses on returning precise results and analyzing them for technical users.
You maintains a professional and straightforward tone, prioritizing technical accuracy and practicality over personalization, ensuring that its responses are detailed, relevant, and valuable to database professionals.
"""


In [24]:
from agency_swarm import Agent

agent_db_env = Agent(name="Database Environment",
                     tools=[InteractWithDataBaseEnv],
                     description="responsible for taking specific tasks and executing them in the database environment (including databases, monitoring systems, etc.), planning and executing specific actions.",
                     instructions=agent_db_env_instructions,
                     files_folder=None)


# CEO Agent


In [25]:
ceo_instructions = """# Instructions for CEO Agent

- Ensure that proposal is send to the user before proceeding with task execution.
- Delegate tasks to appropriate agents, ensuring they align with their expertise and capabilities.
- Clearly define the objectives and expected outcomes for each task.
- Provide necessary context and background information for successful task completion.
- Maintain ongoing communication with agents until complete task execution.
- Review completed tasks to ensure they meet the set objectives.
- Report the results back to the user."""

In [26]:
from agency_swarm import Agent

ceo = Agent(name="CEO",
            description="Responsible for client communication, task planning and management.",
            instructions=ceo_instructions, # can be a file like ./instructions.md
            files_folder=None,
            tools=[])

# Create Agency

In [27]:
agency_manifesto = """# "VRSEN AI" Agency Manifesto

You are a part of a virtual AI development agency called "VRSEN AI"

Your mission is to empower businesses to navigate the AI revolution successfully."""

In [28]:
from agency_swarm import Agency

agency = Agency([
    ceo,
    [ceo, agent_db_env],
], shared_instructions=agency_manifesto)

Updating assistant... Database Environment


In [29]:
agency.demo_gradio(height=900)

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Text sent successfully, check the http://127.0.0.1:5000/task.
Reply: Seq Scan on Departments  (cost=0.00..1.05 rows=1 width=4) (actual time=0.013..0.013 rows=1 loops=1)
  Filter: (DepartmentName = 'IT')
  Rows Removed by Filter: 1
Nested Loop  (cost=0.00..2.10 rows=1 width=15) (actual time=0.027..0.030 rows=2 loops=1)
  ->  Seq Scan on Employees  (cost=0.00..1.03 rows=3 width=19) (actual time=0.010..0.011 rows=3 loops=1)
  ->  Index Scan using department_id_idx on Departments  (cost=0.00..0.35 rows=1 width=4) (actual time=0.005..0.005 rows=1 loops=3)
        Index Cond: (DepartmentID = Employees.DepartmentID)
Total runtime: 0.047 ms

